## DATA FROM THE WEB

In [152]:
from bs4 import BeautifulSoup
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests as rq
import seaborn as sns
import re
sns.set_context('notebook')

In [182]:
def getParamsRequestValue(i_reportmodel_num, i_reportmodel_xsl_num, section, year, semester): 
    r = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter', params={'ww_i_reportmodel' : i_reportmodel_num})
    parameters_map = { 'ww_x_UNITE_ACAD' : section, 'ww_x_PERIODE_ACAD' : year, 'ww_x_PERIODE_PEDAGO' : semester }
    
    soup = BeautifulSoup(r.text, 'html.parser')
    
    ps = {key: next((option.attrs['value'] for option in soup.find('select', {'name': key }) if option.text==value), None) for key, value in parameters_map.items()}
    o = {'ww_i_reportmodel' : i_reportmodel_num, 'ww_b_list' : 1, 'ww_i_reportModelXsl' : i_reportmodel_xsl_num, 'ww_x_HIVERETE' : 'null', 'dummy' : 'ok'}
    d = {**o, **ps}
    r = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter', params=d)
    soup = BeautifulSoup(r.text, 'html.parser')
    for p in soup.find_all('a'):
        if(p.text==', '.join([section, year, semester])):
            arr = re.search('\(\'(.+?)\'\)', p['onclick']).group(1).split("=")
            d[arr[0]] = int(arr[1])
                  
    return d

#getParamsRequestValue(133685247, 133685270, 'Informatique', '2007-2008', 'Bachelor semestre 1')

{'ww_x_HIVERETE': 'null', 'ww_x_PERIODE_PEDAGO': '249108', 'ww_b_list': 1, 'ww_x_GPS': 71297531, 'ww_x_UNITE_ACAD': '249847', 'ww_x_PERIODE_ACAD': '978181', 'ww_i_reportmodel': 133685247, 'dummy': 'ok', 'ww_i_reportModelXsl': 133685270}
